Runner
======
The outermost object that controls the system running. It creates the required objects, loops through the bars and invokes the
EventProcessor on each bar. This is the main object for simulation and live production running.

In [32]:
import pandas as pd
import data as datalib
from database import strategydb, tapdb
import tomahawk.runner as runner
import puma as tw


In [33]:
db_credentials = credentials('test', 'localhost', prefix='db_')
prod_strategydb = strategydb.strategydb_engine(**credentials('test', 'localhost'))
# initialize the Simulation Runner
simrun = runner.SimRunner(**db_credentials)

In [34]:
# setup the market data
simrun.setup_market_data(data_feed='CsvDataFeed', directory='c:/rmbaries/git/pifq/src/puma/data/tests/inst/csv_data_feed')

In [35]:
# access the objects that Runner creates for the Portfolio, Risk, OrderManager and PositionManager
assert isinstance(simrun.risk, tw.Risk)
assert isinstance(simrun.order_manager, tw.OrderManager)
assert isinstance(simrun.position_manager, tw.PositionManager)

In [36]:
# Add the Example Strategy using StrategyDB
strategies = pd.DataFrame({'strategy_id': ['test.example'], 'portfolio_id': 'port_01'})
details = strategydb.get_strategies(prod_strategydb)
strategies = strategies.merge(details, left_on='strategy_id', right_on='strategy_name').drop('strategy_name',axis=1)

simrun.add_strategies(strategies)

In [37]:
# add symbols
symbols = pd.DataFrame({'strategy_id': ['test.example'] * 3, 'product_type': ['stock'] * 3,
                        'symbol_name': ['AAPL', 'MSFT', 'test.sym.3'], 'frequency': ['1min'] * 3})
print(symbols)
simrun.add_symbols(symbols)

In [38]:
# run for a sequence of bars
simrun.run([pd.Timestamp('2010-01-01 09:30:00', tz='America/New_York'), pd.Timestamp('2010-01-01 09:35:00', tz='America/New_York')])

In [39]:
# open order in the OMS, there are none
print(simrun.order_manager.open_orders_df())

In [40]:
# closed orders in the OMS
print(simrun.order_manager.closed_orders_df())

In [41]:
# Trade in PositionManager
print(simrun.position_manager.positions_df)

In [44]:
# Show values persisted into database
tapdb.get_orders_df(simrun.tapdb_engine, 'simulation', pd.Timestamp('2010-01-01 09:35:00', tz='America/New_York'))

In [45]:
tapdb.get_positions_df(simrun.tapdb_engine, 'simulation', pd.Timestamp('2010-01-01 09:35:00', tz='America/New_York'))